In [ ]:
import yfinance as yf
import pandas as pd
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
import keras
from keras import layers

tickerList = ['AAPL', 'AMZN', 'ASML', 'AVGO', 'COST', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA']

# 데이터 로드

- 각 종목코드에 맞는 3년치 감정분석 데이터와 주가 데이터를 가져옵니다.

In [ ]:
# 데이터 전처리 및 로드 함수
def data_load(ticker):
    # 시작 날짜와 종료 날짜 설정
    start_date = '2021-04-05'
    end_date = '2024-05-18'
    
    # MSFT의 경우 시작 날짜를 2021-04-06으로 설정
    if(ticker == 'MSFT'):
        start_date = '2021-04-06'

    # 주식 데이터 다운로드
    datas = yf.download(ticker, start=start_date, end=end_date)

    dic = {
        'ds' : datas.index,
        'y' : datas.Close,
    }

    finance_df = pd.DataFrame( dic )

    #인덱스 초기화(원본 까지 적용)
    finance_df.reset_index( inplace=True )

    del finance_df['Date']

    finance_df.head(5)
    finance_df.tail(5)

    csv_filename = f"./news_data/news_data_{ticker}.csv"
    result_df = pd.read_csv(csv_filename)
    result_df['date'] = pd.to_datetime(result_df['date'])
    result_df = result_df.sort_values('date').reset_index(drop=True)
    date_range = pd.date_range(start=start_date, end=end_date)
    df_date_range = pd.DataFrame(date_range, columns=['date'])
    df_filtered = df_date_range[~df_date_range['date'].isin(finance_df['ds'])]
    common_dates = result_df[result_df['date'].isin(df_filtered['date'])]['date']
    result_filtered = result_df[~result_df['date'].isin(common_dates)]
    result_filtered.reset_index(inplace=True)
    result_filtered = result_filtered.drop(columns=['index'])
    # 데이터 로드
    return finance_df['y'].values, result_filtered['predict'].values

# 학습 준비

- 학습에 필요한 메소드 및 파라미터를 준비합니다.

In [ ]:
# 상태 정의 함수
def get_state(data, t, n, sentiment_scores):
    if t == 0:
        return np.zeros((1, n))  # 초기 상태를 모두 0으로 설정
    
    d = t - n + 1
    if d >= 0:
        block = data[d:t + 1]
    else:
        # 이전 데이터가 없는 경우에는 0으로 채워진 배열을 생성합니다.
        prev_data = np.zeros((n - t - 1,))
        block = np.concatenate([prev_data, data[:t + 1]], axis=0)
    
    # Zero-padding if block length is less than n
    if len(block) < n:
        block = np.concatenate([np.zeros((n - len(block),)), block], axis=0)
    
    res = [block[i + 1] - block[i] for i in range(n - 1)]
    res.append(sentiment_scores[t])
    return np.array([res])

# Actor-Critic 모델 생성 함수 (동일한 모델 구조 사용)
def create_actor_critic_model(input_shape, num_actions):
    inputs = layers.Input(shape=input_shape)
    lstm_layer = layers.LSTM(64, return_sequences=True)(inputs)  # LSTM 레이어 추가
    lstm_layer = layers.LSTM(64)(lstm_layer)  # 추가 LSTM 레이어
    common = layers.Dense(128, activation="relu")(lstm_layer)
    action = layers.Dense(num_actions, activation="softmax")(common)
    critic = layers.Dense(1)(common)
    return keras.Model(inputs=inputs, outputs=[action, critic])

# 하이퍼파라미터 설정 (모델 생성시 사용된 동일한 하이퍼파라미터 사용)
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.1
epsilon_max = 1.0
epsilon_interval = epsilon_max - epsilon_min
num_episodes = 500  # 에피소드 수
n = 10
input_shape = (n, 1)  # 상태 정의 함수의 n 값과 일치해야 함
num_actions = 3  # 예: 매수, 보유, 매도
learning_rate = 0.0001  # 학습률 조정
# discount_factor = 0.99  # 할인 계수 조정

# 모델 및 옵티마이저 생성
model = create_actor_critic_model(input_shape, num_actions)
optimizer = keras.optimizers.Adam(learning_rate)

# 체크포인트 디렉토리
checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# 체크포인트 설정
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{episode}")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

# 학습 시작

- 500 에피소드 동안 학습을 진행합니다.

- 10 에피소드 마다 체크포인트를 저장합니다.

In [13]:
def learning(ticker):
    # 주가 데이터와 감정 분석 값을 준비
    prices, sentiment_scores = data_load(ticker)

    # 평가를 위한 변수 초기화
    total_profits = []
    win_rates = []

    # 학습 루프
    for episode in tqdm(range(1, num_episodes + 1)):  # tqdm 적용
        state = get_state(prices, 0, n, sentiment_scores)
        total_profit = 0
        total_buy = 0
        total_sell = 0
        holdings = 0
        episode_rewards = []
        episode_states = []
        episode_actions = []
        num_buy = 0
        num_sell = 0
        num_trades = 0
        num_wins = 0

        # 시간 단계 루프
        for t in range(len(prices)):
            state_tensor = tf.convert_to_tensor(state, dtype=tf.float32)
            action_probs, critic_value = model(state_tensor)
            action_probs = tf.squeeze(action_probs).numpy()

            if np.random.rand() < epsilon:
                action = np.random.choice(num_actions)
            else:
                action = np.argmax(action_probs)
            
            reward = 0
            if action == 0 and holdings < 50:  # 매수
                holdings += 1
                reward = -prices[t]
                total_buy += prices[t]
                num_buy += 1
                num_trades += 1
            elif action == 2 and holdings > 0:  # 매도
                holdings -= 1
                reward = prices[t] - (total_buy / num_buy) # 평균 매수 가격과 현재 가격의 차이
                total_profit += reward
                total_sell += prices[t]
                num_sell += 1
                num_trades += 1
                if reward > 0:
                    num_wins += 1
            else:  # 보유
                # reward = (holdings * prices[t]) - total_buy
                reward = -holdings

            episode_rewards.append(reward)
            episode_states.append(state)
            episode_actions.append(action)

            next_state = get_state(prices, t, n, sentiment_scores)
            state = next_state

        # 에피소드 종료 시 모든 주식 매도
        while holdings > 0:
            holdings -= 1
            reward = prices[-1] - (total_buy / num_buy) # 평균 매수 가격과 현재 가격의 차이
            total_profit += reward
            total_sell += prices[-1]
            num_sell += 1
            if reward > 0:
                num_wins += 1
            # episode_rewards[-1] += reward

        # 에피소드 종료 후 epsilon 감소
        epsilon -= epsilon_interval / num_episodes
        epsilon = max(epsilon, epsilon_min)

        # Actor-Critic 업데이트
        returns = []
        discounted_sum = 0
        for r in episode_rewards[::-1]: 
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + 1e-8)

        with tf.GradientTape() as tape:
            total_loss = 0
            for i in range(len(episode_rewards)):
                state_tensor = tf.convert_to_tensor(episode_states[i], dtype=tf.float32)
                action_probs, critic_value = model(state_tensor)
                critic_value = tf.squeeze(critic_value)

                advantage = returns[i] - critic_value
                action_onehot = tf.one_hot(episode_actions[i], num_actions)
                actor_loss = -tf.math.log(tf.reduce_sum(action_probs * action_onehot)) * advantage
                critic_loss = advantage ** 2
                total_loss += actor_loss + critic_loss
            
            grads = tape.gradient(total_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        # 에피소드 결과 저장 및 출력
        win_rate = num_wins / num_trades if num_trades > 0 else 0
        total_profits.append(total_profit)
        win_rates.append(win_rate)
        print(f"Episode {episode}: Total Profit: {total_profit}, Win Rate: {win_rate}")

        # 10 에피소드마다 체크포인트 저장
        if episode % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix.format(episode=episode))

    # 학습 완료 후 성과 평가
    average_profit = np.mean(total_profits)
    average_win_rate = np.mean(win_rates)
    sharpe_ratio = np.mean(total_profits) / (np.std(total_profits) + 1e-8)
    print(f"Average Profit: {average_profit}, Average Win Rate: {average_win_rate}, Sharpe Ratio: {sharpe_ratio}")


2024-06-12 10:07:46.016099: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-06-12 10:07:46.016118: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-06-12 10:07:46.016124: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-06-12 10:07:46.016145: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-12 10:07:46.016156: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
  0%|          | 1/500 [00:29<4:03:13, 29.25s/it]

Episode 1: Total Profit: 3190.5183226694417, Win Rate: 0.4051724137931034


  0%|          | 2/500 [00:57<3:59:07, 28.81s/it]

Episode 2: Total Profit: 3181.1854427801077, Win Rate: 0.38943248532289626


  1%|          | 3/500 [01:26<3:57:13, 28.64s/it]

...

Episode 498: Total Profit: 2844.6916435807084, Win Rate: 0.39814814814814814


100%|█████████▉| 499/500 [4:05:50<01:00, 60.21s/it]

Episode 499: Total Profit: 3255.9326965249197, Win Rate: 0.38755020080321284


100%|██████████| 500/500 [4:07:27<00:00, 29.69s/it]

Episode 500: Total Profit: 3686.765491107997, Win Rate: 0.4050880626223092
Average Profit: 2450.5131152931112, Average Win Rate: 0.38669222889547594, Sharpe Ratio: 4.968962504488752
